In [1]:
from obspy import read
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.core import AttribDict
from obspy.io.sac import SACTrace
import numpy as np
import matplotlib.pylab as plt
import os
import pandas as pd
import shutil
from tqdm import tqdm
import warnings
from obspy import read_inventory, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site

In [3]:
# check the sample rate of a sac file and downsample
sr_new = 1
abnormal_sac_files = ['/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/island_networks/output_island/IU/IU/PTCN/PTCN.2002.043.00.00.00.BHZ.sac',
                      '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/island_networks/output_island/IU/IU/AFI/AFI.2001.335.00.00.00.BHZ.sac',
                     '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/island_networks/output_island/II/II/MSVF/MSVF.2018.106.00.00.00.BHZ.sac',
                     '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/IU/PTCN_IU/PTCN_IU.2001.335.00.00.00.BHZ.sac']

# See the date of the first file in abnormal_sac_files to enter 'time' below.
st = read(abnormal_sac_files[3],format="SAC")
sr = st[0].stats.sampling_rate
st = st.decimate(factor=int(sr/sr_new), no_filter=True) # downsample
print(st)

1 Trace(s) in Stream:
IU.PTCN.00.BHZ | 2001-12-01T00:00:00.036000Z - 2001-12-01T23:59:59.036000Z | 1.0 Hz, 86400 samples


In [2]:
# downsample all sac files in folders
sr_new = 1.0
#the folder where all downloaded files in a network located
folder_path = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/Z4'

#the folder that store all files in a network after removing response. You need to create this folder first.
output_folder = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/Z4'

sac_files = []

# loop through all station folders in the network folder
for station_folder in os.listdir(folder_path):
    station_folder_path = os.path.join(folder_path, station_folder)
    for root, dirs, files in os.walk(station_folder_path):
        # loop through all files in the station folder
        for file in files:
            if file.endswith(".sac"):
                # add file to sac_files list
                sac_files.append(os.path.join(root, file))

# count files that need to remove response
count = len(sac_files)
print(count)

#progress bar
pbar = tqdm(total=len(sac_files))

# list to store files with errors
abnormal_sac_files = []

# Load all SAC files in sac_files list using ObsPy
for sac_file in sac_files:
    try:
        st = read(sac_file)
        sr = st[0].stats.sampling_rate
        st = st.filter('lowpass', freq=0.4*sr_new, zerophase=True) # anti-alias filter
        st = st.filter('highpass', freq=1/60/60, zerophase=True) # Remove daily oscillations
        st = st.decimate(factor=int(sr/sr_new), no_filter=True) # downsample
        st = st.detrend(type='demean')
        st = st.detrend(type='linear')
        st.taper(type="cosine",max_percentage=0.05)
        print(st)

        # Create output station folders
        relative_path = os.path.relpath(os.path.dirname(sac_file), folder_path)
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)

        # Generate output file path
        # Output files will have the same names as the original files
        output_file_name = os.path.basename(sac_file)
        output_file_path = os.path.join(output_subfolder, output_file_name)

        # Write the modified stream to the output file path
        st.write(output_file_path, format="SAC")
        pbar.update(1)
        
    #If there's an error when removing response, skip that file and continue            
    except ValueError as e:
        print(f"Skipping '{sac_file}': {str(e)}")
        abnormal_sac_files.append(sac_file)
        
        #If there's an error when removing response, copy the file as it is to new folder
        #error_file_path = os.path.join(output_subfolder, output_file_name)
        #shutil.copy(sac_file, error_file_path)
        #print(f"Original file copied to '{error_file_path}'")
        continue
                 
# Close the progress bar
pbar.close()

print("Files with errors:")
print(abnormal_sac_files)

3775


  0%|          | 3/3775 [00:10<2:58:00,  2.83s/it] 

1 Trace(s) in Stream:
Z4.CP08..BHZ | 2013-02-10T00:00:00.005400Z - 2013-02-10T23:59:59.005400Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BH2 | 2013-03-06T00:00:00.018100Z - 2013-03-06T23:59:59.018100Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BHZ | 2012-09-30T00:00:00.014500Z - 2012-09-30T23:59:59.014500Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BHZ | 2012-08-16T00:00:00.003300Z - 2012-08-16T23:59:59.003300Z | 1.0 Hz, 86400 samples


  0%|          | 9/3775 [00:11<33:52,  1.85it/s]  

1 Trace(s) in Stream:
Z4.CP08..BHZ | 2012-08-23T00:00:00.002900Z - 2012-08-23T23:59:59.002900Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BHZ | 2013-02-03T00:00:00.005900Z - 2013-02-03T23:59:59.005900Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BH2 | 2012-10-28T00:00:00.012600Z - 2012-10-28T23:59:59.012600Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BH2 | 2012-07-24T00:00:00.012000Z - 2012-07-24T23:59:59.012000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP08..BH2 | 2012-05-17T00:00:00.009600Z - 2012-05-17T23:59:59.009600Z | 1.0 Hz, 86400 samples


...

1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-11-14T00:00:00.021400Z - 2012-11-14T23:59:59.021400Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH1 | 2013-02-24T00:00:00.004700Z - 2013-02-24T23:59:59.004700Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-08-28T00:00:00.003300Z - 2012-08-28T23:59:59.003300Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-05-16T00:00:00.020800Z - 2012-05-16T23:59:59.020800Z | 1.0 Hz, 86400 samples


100%|█████████▉| 3772/3775 [03:36<00:00, 18.93it/s]

1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-09-13T00:00:00.021700Z - 2012-09-13T23:59:59.021700Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BHZ | 2012-12-11T00:00:00.010400Z - 2012-12-11T23:59:59.010400Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH2 | 2013-02-20T00:00:00.006300Z - 2013-02-20T23:59:59.006300Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-07-27T00:00:00.016400Z - 2012-07-27T23:59:59.016400Z | 1.0 Hz, 86400 samples


100%|██████████| 3775/3775 [03:36<00:00, 17.43it/s]

1 Trace(s) in Stream:
Z4.CP09..BHZ | 2012-11-11T00:00:00.022600Z - 2012-11-11T23:59:59.022600Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
Z4.CP09..BH1 | 2012-10-21T00:00:00.006200Z - 2012-10-21T23:59:59.006200Z | 1.0 Hz, 86400 samples
Files with errors:
[]


In [62]:
# If you want to enter abnormal_sac_files by hand
abnormal_sac_files = ['/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/island_networks/output_island/IU/IU/PTCN/PTCN.2002.043.00.00.00.BHZ.sac',
                      '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/IU/AFI/AFI.2009.325.00.00.00.BHZ.sac',
                     '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/II/MSVF/MSVF.2001.348.00.00.00.BHZ.sac',
                     '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/G/NOUC/NOUC.2001.349.00.00.00.BHZ.sac']
sr_new=1.0
# See the date of the first file in abnormal_sac_files to enter 'time' below.
st = read(abnormal_sac_files[1],format="SAC")
# sr = st[0].stats.sampling_rate
# print(sr)
# st = st.filter('lowpass', freq=0.4*sr_new, zerophase=True) # anti-alias filter
# st = st.filter('highpass', freq=1/60/60, zerophase=True) # Remove daily oscillations
# st = st.decimate(factor=int(sr/sr_new), no_filter=True) # downsample

print(st)

1 Trace(s) in Stream:
IU.AFI.10.BHZ | 2009-11-21T00:00:00.010645Z - 2009-11-21T23:59:59.010645Z | 1.0 Hz, 86400 samples


In [59]:
#the folder where all downloaded files in a network located
folder_path = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/YL'

#the folder that store all files in a network after removing response. You need to create this folder first.
output_folder = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/IU'

sac_files = []

# loop through all station folders in the network folder
for station_folder in os.listdir(folder_path):
    station_folder_path = os.path.join(folder_path, station_folder)
    for root, dirs, files in os.walk(station_folder_path):
        # loop through all files in the station folder
        for file in files:
            if file.endswith(".sac"):
                # add file to sac_files list
                sac_files.append(os.path.join(root, file))

# count files that need to remove response
count = len(sac_files)
print(count)

#progress bar
pbar = tqdm(total=len(sac_files))

# list to store files with errors
abnormal_sac_files = []

# Load all SAC files in sac_files list using ObsPy
for sac_file in sac_files:
    st = read(sac_file)
    print(st)

7740



  0%|          | 0/13713 [00:36<?, ?it/s][A

1 Trace(s) in Stream:
YL.B02..BHZ | 2010-08-01T00:00:00.020000Z - 2010-08-01T23:59:59.020000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-01-10T00:00:00.006000Z - 2010-01-10T23:59:59.006000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-09-12T00:00:00.007000Z - 2010-09-12T23:59:59.007000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-04-21T00:00:00.015000Z - 2010-04-21T23:59:59.015000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2009-12-03T00:00:00.003000Z - 2009-12-03T23:59:59.003000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-06-21T00:00:00.004000Z - 2010-06-21T23:59:59.004000Z | 1.0 Hz, 86400 samples


1 Trace(s) in Stream:
YL.B02..BHZ | 2010-07-04T00:00:00.003000Z - 2010-07-04T23:59:59.003000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-09-22T00:00:00.018000Z - 2010-09-22T23:59:59.018000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-07-05T00:00:00.024000Z - 2010-07-05T23:59:59.024000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-10-01T00:00:00.009000Z - 2010-10-01T23:59:59.009000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-10-21T00:00:00.006000Z - 2010-10-21T23:59:59.006000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-09-20T00:00:00.001000Z - 2010-09-20T23:59:59.001000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-11-01T00:00:00.014000Z - 2010-11-01T23:59:59.014000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-06-15T00:00:00.002000Z - 2010-06-15T23:59:59.002000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2009-11-27T00:00:00.

1 Trace(s) in Stream:
YL.B02..BHZ | 2009-12-06T00:00:00.016000Z - 2009-12-06T23:59:59.016000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-08-30T00:00:00.007000Z - 2010-08-30T23:59:59.007000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-05-04T00:00:00.014000Z - 2010-05-04T23:59:59.014000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-06-04T00:00:00.019000Z - 2010-06-04T23:59:59.019000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-06-01T00:00:00.006000Z - 2010-06-01T23:59:59.006000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-08-21T00:00:00.017000Z - 2010-08-21T23:59:59.017000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-07-17T00:00:00.003000Z - 2010-07-17T23:59:59.003000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-05-28T00:00:00.021000Z - 2010-05-28T23:59:59.021000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-11-18T00:00:00.

1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-22T00:00:00.006000Z - 2010-03-22T23:59:59.006000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-18T00:00:00.021000Z - 2010-03-18T23:59:59.021000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-07-18T00:00:00.024000Z - 2010-07-18T23:59:59.024000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-04-07T00:00:00.019000Z - 2010-04-07T23:59:59.019000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2009-12-29T00:00:00.002000Z - 2009-12-29T23:59:59.002000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-05-08T00:00:00.024000Z - 2010-05-08T23:59:59.024000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-08-22T00:00:00.013000Z - 2010-08-22T23:59:59.013000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-08T00:00:00.010000Z - 2010-03-08T23:59:59.010000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-04-11T00:00:00.

1 Trace(s) in Stream:
YL.B02..BHZ | 2010-05-24T00:00:00.012000Z - 2010-05-24T23:59:59.012000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-13T00:00:00.016000Z - 2010-03-13T23:59:59.016000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-01-02T00:00:00.012000Z - 2010-01-02T23:59:59.012000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-02-21T00:00:00.018000Z - 2010-02-21T23:59:59.018000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-05-16T00:00:00.018000Z - 2010-05-16T23:59:59.018000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-09T00:00:00.006000Z - 2010-03-09T23:59:59.006000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-09-23T00:00:00.015000Z - 2010-09-23T23:59:59.015000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-04T00:00:00.001000Z - 2010-03-04T23:59:59.001000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-07-15T00:00:00.

1 Trace(s) in Stream:
YL.B02..BHZ | 2010-02-25T00:00:00.003000Z - 2010-02-25T23:59:59.003000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-01-14T00:00:00.015000Z - 2010-01-14T23:59:59.015000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-04-08T00:00:00.015000Z - 2010-04-08T23:59:59.015000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-06-27T00:00:00.005000Z - 2010-06-27T23:59:59.005000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-01-28T00:00:00.011000Z - 2010-01-28T23:59:59.011000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-29T00:00:00.004000Z - 2010-03-29T23:59:59.004000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-09-17T00:00:00.013000Z - 2010-09-17T23:59:59.013000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-04-15T00:00:00.013000Z - 2010-04-15T23:59:59.013000Z | 1.0 Hz, 86400 samples
1 Trace(s) in Stream:
YL.B02..BHZ | 2010-03-03T00:00:00.

KeyboardInterrupt: 